## Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

In [1]:


# importing old libraries (also used in the two first parts)
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs



In [3]:
# importing new libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
 # map rendering library

print('Libraries imported.')

Libraries imported.


In [15]:
df_toronto = pd.read_csv('toronto_part2.csv')
df_toronto.head(7)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
1,M1B\n,Scarborough\n,"Malvern, Rouge",43.81153,-79.19552
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill",43.76575,-79.17520
4,M1G\n,Scarborough\n,Woburn,43.76820,-79.21761
5,M1H\n,Scarborough\n,Cedarbrae,43.76969,-79.23944
6,M1J\n,Scarborough\n,Scarborough Village,43.74309,-79.23526


In [16]:


# @hiddel_cell
CLIENT_ID = 'QDZ1XDM3XJOJKISZG5R1X5A3B54HNAPLRL1JXFUBGZUXUZKM' # your Foursquare ID
CLIENT_SECRET = 'EWNWZB4APZNV3YR5DCCHRPQ2UGCJYW31UVAYD1RSNGIKYLYT' # your Foursquare Secret
VERSION = '20190530' # Foursquare API version



In [17]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough\n'].reset_index(drop=True)
scarborough_data.head(7)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B\n,Scarborough\n,"Malvern, Rouge",43.81153,-79.19552
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill",43.76575,-79.17520
3,M1G\n,Scarborough\n,Woburn,43.76820,-79.21761
4,M1H\n,Scarborough\n,Cedarbrae,43.76969,-79.23944
5,M1J\n,Scarborough\n,Scarborough Village,43.74309,-79.23526
6,M1K\n,Scarborough\n,"Kennedy Park, Ionview, East Birchmount Park",43.72861,-79.26367


### Geographical coordinate of Scarborough 

In [18]:
address_scar = 'Scarborough\n,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


### Get the top 100 venues in the neighborhood 'Steeles West', from Scarborough

In [23]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Malvern, Rouge" are 43.81153000000007, -79.19551999999999.


In [24]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f115d6c79fcaa78931f518e'},
 'response': {'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 39,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775537,
          'lng': -79.256833}],
        'distance

In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Store,Clothing Store,43.776012,-79.258334
3,Hot Topic,Clothing Store,43.775450,-79.257929
4,DAVIDsTEA,Tea Room,43.776320,-79.258688
5,Tommy Hilfiger,Clothing Store,43.776015,-79.257369
6,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
7,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
8,Shoppers Drug Mart,Pharmacy,43.773305,-79.251662
9,St. Louis Bar & Grill,Bar,43.774157,-79.253808


In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

39 venues were returned by Foursquare.


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Get venues for each neighborhood in Scarborough

In [32]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [33]:
scarborough_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.81153,-79.19552,R & K Woodworking Specialists Inc,43.808233,-79.196857,Construction & Landscaping
1,"Malvern, Rouge",43.81153,-79.19552,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
2,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871,Organized By Catherine,43.782117,-79.155239,Moving Target
4,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871,Royal Canadian Legion,43.782533,-79.163085,Bar
5,"Guildwood, Morningside, West Hill",43.76575,-79.17520,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
6,"Guildwood, Morningside, West Hill",43.76575,-79.17520,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center
7,"Guildwood, Morningside, West Hill",43.76575,-79.17520,Heron Park,43.769327,-79.177201,Park
8,Woburn,43.76820,-79.21761,Starbucks,43.770037,-79.221156,Coffee Shop
9,Woburn,43.76820,-79.21761,cheapOseo,43.766042,-79.218539,Business Service


In [34]:
scarborough_venues.tail(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
75,"Steeles West, L'Amoreaux West",43.80052,-79.32074,RBC Royal Bank,43.798236,-79.317952,Bank
76,"Steeles West, L'Amoreaux West",43.80052,-79.32074,KFC,43.798938,-79.318854,Fast Food Restaurant
77,"Steeles West, L'Amoreaux West",43.80052,-79.32074,Shoppers Drug Mart,43.799966,-79.317985,Pharmacy
78,"Steeles West, L'Amoreaux West",43.80052,-79.32074,Tim Hortons,43.799102,-79.318715,Coffee Shop
79,"Steeles West, L'Amoreaux West",43.80052,-79.32074,McDonald's,43.798249,-79.318167,Fast Food Restaurant
80,"Steeles West, L'Amoreaux West",43.80052,-79.32074,Pizza Pizza,43.797909,-79.318113,Pizza Place
81,"Steeles West, L'Amoreaux West",43.80052,-79.32074,Eggsmart,43.796375,-79.318681,Breakfast Spot
82,"Steeles West, L'Amoreaux West",43.80052,-79.32074,Dollarama,43.798394,-79.318453,Discount Store
83,"Steeles West, L'Amoreaux West",43.80052,-79.32074,Metro,43.797830,-79.318492,Supermarket
84,"Steeles West, L'Amoreaux West",43.80052,-79.32074,Warden Park,43.804789,-79.320449,Other Great Outdoors


In [35]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,8,8,8,8,8,8
"Birch Cliff, Cliffside West",5,5,5,5,5,5
Cedarbrae,1,1,1,1,1,1
"Clarks Corners, Tam O'Shanter, Sullivan",8,8,8,8,8,8
"Cliffside, Cliffcrest, Scarborough Village West",10,10,10,10,10,10
"Dorset Park, Wexford Heights, Scarborough Town Centre",4,4,4,4,4,4
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",3,3,3,3,3,3
"Kennedy Park, Ionview, East Birchmount Park",7,7,7,7,7,7


In [36]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 50 uniques categories.


In [37]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,Auto Garage,Badminton Court,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Business Service,Café,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,History Museum,Home Service,Ice Cream Shop,Intersection,Korean Restaurant,Light Rail Station,Liquor Store,Metro Station,Moving Target,Other Great Outdoors,Park,Pharmacy,Pizza Place,Playground,Pool,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Thai Restaurant,Train Station
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
scarb_onehot.shape

(85, 51)

In [39]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,Auto Garage,Badminton Court,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Business Service,Café,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,History Museum,Home Service,Ice Cream Shop,Intersection,Korean Restaurant,Light Rail Station,Liquor Store,Metro Station,Moving Target,Other Great Outdoors,Park,Pharmacy,Pizza Place,Playground,Pool,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Thai Restaurant,Train Station
0,Agincourt,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.125,0.000,0.000,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.125,0.0,0.000,0.0,0.125,0.0,0.0,0.125,0.125,0.0,0.125,0.125,0.000,0.0
1,"Birch Cliff, Cliffside West",0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.000,0.000,0.2,0.0,0.0,0.0,0.000,0.000,0.000,0.2,0.00,0.000,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.200,0.0,0.000,0.000,0.000,0.0
2,Cedarbrae,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000,0.0,0.000,1.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.125,0.0,0.0,0.0,0.0,0.000,0.125,0.125,0.0,0.00,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000,0.0,0.125,0.0,0.000,0.0,0.0,0.125,0.000,0.0,0.000,0.000,0.125,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.100,0.0,0.0,0.0,0.0,0.100,0.000,0.000,0.0,0.00,0.000,0.0,0.0,0.0,0.1,0.0,0.0,0.2,0.0,0.0,0.00,0.1,0.0,0.0,0.0,0.000,0.1,0.100,0.0,0.000,0.1,0.1,0.000,0.000,0.0,0.000,0.000,0.000,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.25,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.0
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000,0.2,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.000,0.100,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.00,0.000,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.00,0.0,0.1,0.0,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.1,0.000,0.000,0.000,0.0


## Get top 10 venues per neighborhood

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Sushi Restaurant,Supermarket,Badminton Court,Skating Rink,Shopping Mall,Pool,Park,Discount Store,Train Station,Department Store
1,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Gym,Train Station,Golf Course,Gift Shop,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Playground,Train Station,Coffee Shop,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Chinese Restaurant,Pizza Place,Golf Course,Fried Chicken Joint,Fast Food Restaurant,Coffee Shop,Thai Restaurant,Shopping Mall,Breakfast Spot,Convenience Store
4,"Cliffside, Cliffcrest, Scarborough Village West",Ice Cream Shop,Coffee Shop,Hardware Store,Discount Store,Sandwich Place,Restaurant,Pizza Place,Pharmacy,Liquor Store,Train Station
5,"Dorset Park, Wexford Heights, Scarborough Town...",Bakery,Gift Shop,Light Rail Station,Train Station,College Stadium,Golf Course,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store
6,"Golden Mile, Clairlea, Oakridge",Intersection,Bakery,Coffee Shop,Bus Line,Metro Station,Bus Station,Gym,Soccer Field,Breakfast Spot,Department Store
7,"Guildwood, Morningside, West Hill",Gym / Fitness Center,Park,Construction & Landscaping,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store
8,"Kennedy Park, Ionview, East Birchmount Park",Coffee Shop,Convenience Store,Light Rail Station,Bus Station,Department Store,College Stadium,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint
9,"Malvern, Rouge",Home Service,Construction & Landscaping,Train Station,College Stadium,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store


## Run k-means to cluster the neighborhoods into 5 clusters

In [42]:
 #import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(14)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([1, 1, 2, 1, 1, 1, 1, 3, 1, 3], dtype=int32)

In [44]:


scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarb_merged



,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B\n,Scarborough\n,"Malvern, Rouge",43.81153,-79.19552,1,Home Service,Construction & Landscaping,Train Station,College Stadium,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871,1,History Museum,Bar,Moving Target,Train Station,Construction & Landscaping,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill",43.76575,-79.17520,2,Gym / Fitness Center,Park,Construction & Landscaping,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store
3,M1G\n,Scarborough\n,Woburn,43.76820,-79.21761,1,Park,Korean Restaurant,Business Service,Coffee Shop,College Stadium,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
4,M1H\n,Scarborough\n,Cedarbrae,43.76969,-79.23944,1,Playground,Train Station,Coffee Shop,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store
5,M1J\n,Scarborough\n,Scarborough Village,43.74309,-79.23526,1,Ice Cream Shop,Train Station,Restaurant,College Stadium,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store
6,M1K\n,Scarborough\n,"Kennedy Park, Ionview, East Birchmount Park",43.72861,-79.26367,1,Coffee Shop,Convenience Store,Light Rail Station,Bus Station,Department Store,College Stadium,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint
7,M1L\n,Scarborough\n,"Golden Mile, Clairlea, Oakridge",43.71406,-79.28412,3,Intersection,Bakery,Coffee Shop,Bus Line,Metro Station,Bus Station,Gym,Soccer Field,Breakfast Spot,Department Store
8,M1M\n,Scarborough\n,"Cliffside, Cliffcrest, Scarborough Village West",43.72360,-79.23496,1,Ice Cream Shop,Coffee Shop,Hardware Store,Discount Store,Sandwich Place,Restaurant,Pizza Place,Pharmacy,Liquor Store,Train Station
9,M1N\n,Scarborough\n,"Birch Cliff, Cliffside West",43.69539,-79.26194,3,College Stadium,General Entertainment,Skating Rink,Café,Gym,Train Station,Golf Course,Gift Shop,Fried Chicken Joint,Fast Food Restaurant


In [45]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough\n,0,Bakery,Gift Shop,Light Rail Station,Train Station,College Stadium,Golf Course,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store


In [46]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough\n,1,Home Service,Construction & Landscaping,Train Station,College Stadium,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store
1,Scarborough\n,1,History Museum,Bar,Moving Target,Train Station,Construction & Landscaping,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
3,Scarborough\n,1,Park,Korean Restaurant,Business Service,Coffee Shop,College Stadium,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
4,Scarborough\n,1,Playground,Train Station,Coffee Shop,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store
5,Scarborough\n,1,Ice Cream Shop,Train Station,Restaurant,College Stadium,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store
6,Scarborough\n,1,Coffee Shop,Convenience Store,Light Rail Station,Bus Station,Department Store,College Stadium,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint
8,Scarborough\n,1,Ice Cream Shop,Coffee Shop,Hardware Store,Discount Store,Sandwich Place,Restaurant,Pizza Place,Pharmacy,Liquor Store,Train Station
11,Scarborough\n,1,Auto Garage,College Stadium,Grocery Store,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store
12,Scarborough\n,1,Sushi Restaurant,Supermarket,Badminton Court,Skating Rink,Shopping Mall,Pool,Park,Discount Store,Train Station,Department Store
13,Scarborough\n,1,Chinese Restaurant,Pizza Place,Golf Course,Fried Chicken Joint,Fast Food Restaurant,Coffee Shop,Thai Restaurant,Shopping Mall,Breakfast Spot,Convenience Store


In [47]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough\n,2,Gym / Fitness Center,Park,Construction & Landscaping,Golf Course,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store


In [48]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough\n,3,Intersection,Bakery,Coffee Shop,Bus Line,Metro Station,Bus Station,Gym,Soccer Field,Breakfast Spot,Department Store
9,Scarborough\n,3,College Stadium,General Entertainment,Skating Rink,Café,Gym,Train Station,Golf Course,Gift Shop,Fried Chicken Joint,Fast Food Restaurant


In [49]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Scarborough\n,4,Fast Food Restaurant,Chinese Restaurant,Other Great Outdoors,Breakfast Spot,Discount Store,Coffee Shop,Pharmacy,Pizza Place,Grocery Store,Sandwich Place
